In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
project_id = 'bigquerycourse-464720'

In [52]:
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


#Active Customers Calculation


## 🔍 Exploring and Scoping the Dataset

>Let's explore Users and Orders table

In [ ]:
%%bigquery --project {project_id}
SELECT
  *
FROM
  `bigquery-public-data.thelook_ecommerce.users`
LIMIT
  5
# This is a table of All Unique Users with properities like id and others.
# But, we need aditional data because theirs no columns related to orders for each user.

Query is running:   0%|          |

Downloading:   0%|          |

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at,user_geom
0,6769,Timothy,Herman,timothyherman@example.com,37,M,Acre,66068 James Gateway,69980-000,null,Brasil,-8.065346,-72.870949,Search,2024-09-28 13:24:00+00:00,POINT(-72.87094866 -8.065346116)
1,26739,Alexis,Hernandez,alexishernandez@example.org,31,F,Acre,3348 Brenda Mills,69980-000,null,Brasil,-8.065346,-72.870949,Email,2025-08-15 01:19:00+00:00,POINT(-72.87094866 -8.065346116)
2,75610,Kristin,Chavez,kristinchavez@example.org,63,F,Acre,6872 Virginia Curve,69980-000,null,Brasil,-8.065346,-72.870949,Facebook,2024-11-15 10:03:00+00:00,POINT(-72.87094866 -8.065346116)
3,63702,Lance,Gordon,lancegordon@example.net,57,M,Acre,860 Ramirez Extensions Apt. 764,69980-000,null,Brasil,-8.065346,-72.870949,Search,2020-08-20 15:47:00+00:00,POINT(-72.87094866 -8.065346116)
4,771,Thomas,Castillo,thomascastillo@example.net,44,M,Acre,5549 Gonzales Route,69980-000,null,Brasil,-8.065346,-72.870949,Search,2022-12-22 09:22:00+00:00,POINT(-72.87094866 -8.065346116)


In [35]:
%%bigquery --project {project_id}
SELECT
  *
from
  `bigquery-public-data.thelook_ecommerce.orders`
LIMIT
  5
# This is a table of All UNIQUE orders that Users with other details like order_id and user_id.
# The good is that we have the user_id. With this we can relate the USERs table with the Order table.

Query is running:   0%|          |

Downloading:   0%|          |

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,21,15,Cancelled,F,2025-05-27 02:03:00+00:00,NaT,NaT,NaT,2
1,30,22,Cancelled,F,2025-06-20 10:39:00+00:00,NaT,NaT,NaT,1
2,40,26,Cancelled,F,2021-04-28 13:41:00+00:00,NaT,NaT,NaT,2
3,45,29,Cancelled,F,2023-06-08 03:09:00+00:00,NaT,NaT,NaT,1
4,63,41,Cancelled,F,2025-08-27 07:25:00+00:00,NaT,NaT,NaT,1


> Before moving foward, the KPI to be calculated need to be in a certain period (quarter). Now let's know the time period of the Order Table and select one Quarter.

In [40]:
#We realize that is a created_at column. To move foward
#we need to know the time period.

%%bigquery --project {project_id}
SELECT
  min(date(created_at)) as First_Date,
  max(date(created_at)) as Last_Date,
  max(date(created_at))-min(date(created_at)) as Time_Period,
  DATE_DIFF(max(date(created_at)), min(date(created_at)), DAY) as Time_Period_Days,
from
  `bigquery-public-data.thelook_ecommerce.orders`

#The records go from 2019-01-13 to 2025-09-02 and includes 2424 days of transactions.
#note that using DATE_DIFF will return the bays between in a integer data type.

Query is running:   0%|          |

Downloading:   0%|          |

,First_Date,Last_Date,Time_Period,Time_Period_Days
0,2019-01-13,2025-09-02,"<DateOffset: days=2424, microseconds=0, months...",2424


> Now that we have confirmed the date range of the data, lets select the first quarter of 2025. We will run just to checking pourposuses.

In [44]:
#Lets check the info
%%bigquery --project {project_id}
SELECT
  min(date(created_at)) as Order_Date,
  max(date(created_at)) as Order_Date,
  DATE_DIFF(max(date(created_at)), min(date(created_at)), DAY) as Time_Period_Days,
from
  `bigquery-public-data.thelook_ecommerce.orders`
where
  date(created_at) between '2025-01-01' and '2025-03-31'
#

Query is running:   0%|          |

Downloading:   0%|          |

,Order_Date,Order_Date_1,Time_Period_Days
0,2025-01-01,2025-03-31,89


## 💻 Creating the Query

>Now that we have confirmed the date range of the data , we can move on to creating a query that identifies active users within the first quarter of 2025.

In [49]:
%%bigquery --project {project_id}
SELECT
  distinct (U.id) AS User_ID,
  COUNT(O.order_id) ORDERS
FROM
  bigquery-public-data.thelook_ecommerce.users U
LEFT JOIN
  bigquery-public-data.thelook_ecommerce.orders O
ON
  U.id = O.user_id
WHERE
  O.order_id IS NOT NULL and date(O.created_at) between '2025-01-01' and '2025-03-31'
GROUP BY
  User_ID
ORDER BY
  User_ID asc

Query is running:   0%|          |

Downloading:   0%|          |

,User_ID,ORDERS
0,6,1
1,10,1
2,13,1
3,21,1
4,22,1
...,...,...
10445,99969,1
10446,99977,1
10447,99981,2
10448,99985,1


In [51]:
%%bigquery --project {project_id}
SELECT
    COUNT(User_ID) AS Total_Users
FROM (
    SELECT
  distinct (U.id) AS User_ID,
  COUNT(O.order_id) ORDERS
FROM
  bigquery-public-data.thelook_ecommerce.users U
LEFT JOIN
  bigquery-public-data.thelook_ecommerce.orders O
ON
  U.id = O.user_id
WHERE
  O.order_id IS NOT NULL
  AND date(O.created_at) between '2025-01-01' and '2025-03-31'
GROUP BY
  User_ID
ORDER BY
  User_ID asc
) AS subquery_result

Query is running:   0%|          |

Downloading:   0%|          |

,Total_Users
0,10450
